In [1]:
## Import necessary packages
%matplotlib inline
import os
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import linregress

In [2]:
# Load in the Austin crime dataset
# df = os.path.join('Resources', 'Crime_Reports.csv')

datafile = '../Resources/Crime_Reports.csv'

In [3]:
data = pd.read_csv(datafile)
data.head()

,Incident Number,Highest Offense Description,Highest Offense Code,Family Violence,Occurred Date Time,Occurred Date,Occurred Time,Report Date Time,Report Date,Report Time,...,Census Tract,Clearance Status,Clearance Date,UCR Category,Category Description,X-coordinate,Y-coordinate,Latitude,Longitude,Location
0,2.017115e+08,AGG ASLT STRANGLE/SUFFOCATE,410,Y,1/1/2017 10:30,1/1/2017,1030,1/1/2017 11:58,1/1/2017,1158,...,NaN,C,1/10/2017,13A,Aggravated Assault,3130704.0,3130704.0,30.359794,-97.688358,"(30.35979391, -97.688358)"
1,2.017100e+08,AGG ASSAULT,402,N,1/1/2017 0:07,1/1/2017,7,1/1/2017 0:07,1/1/2017,7,...,NaN,NaN,NaN,13A,Aggravated Assault,3114768.0,3114768.0,30.268162,-97.741362,"(30.26816195, -97.74136237)"
2,2.017109e+08,AGG ASSAULT,402,N,1/1/2017 4:20,1/1/2017,420,1/1/2017 4:20,1/1/2017,420,...,NaN,N,3/8/2017,13A,Aggravated Assault,3131193.0,3131193.0,30.318604,-97.687937,"(30.3186038, -97.68793685)"
3,2.017503e+10,AGG ASSAULT FAM/DATE VIOLENCE,402,Y,1/1/2017 0:00,1/1/2017,0,8/10/2017 8:30,8/10/2017,830,...,NaN,O,8/17/2017,13A,Aggravated Assault,3099695.0,3099695.0,30.196029,-97.791021,"(30.1960291, -97.79102086)"
4,2.019239e+10,AGG FORCED SODOMY,1718,N,1/1/2017 21:29,1/1/2017,2129,8/27/2019 21:29,8/27/2019,2129,...,NaN,C,10/29/2019,11B,Rape,0.0,0.0,NaN,NaN,NaN


In [4]:
data.columns

Index(['Incident Number', 'Highest Offense Description',
       'Highest Offense Code', 'Family Violence', 'Occurred Date Time',
       'Occurred Date', 'Occurred Time', 'Report Date Time', 'Report Date',
       'Report Time', 'Location Type', 'Address', 'Zip Code',
       'Council District', 'APD Sector', 'APD District', 'PRA', 'Census Tract',
       'Clearance Status', 'Clearance Date', 'UCR Category',
       'Category Description', 'X-coordinate', 'Y-coordinate', 'Latitude',
       'Longitude', 'Location'],
      dtype='object')

In [5]:
Newdata = data.drop(columns=["Incident Number", "Highest Offense Code", "Family Violence", 'Occurred Date', 'Report Date Time', 'Report Date',
       'Report Time', 'Address', 'Council District', 'APD Sector', 'Clearance Status', 'Clearance Date', 'UCR Category', 'X-coordinate', 'Y-coordinate', 'Latitude', 'Longitude'])
Newdata.head()

,Highest Offense Description,Occurred Date Time,Occurred Time,Location Type,Zip Code,APD District,PRA,Census Tract,Category Description,Location
0,AGG ASLT STRANGLE/SUFFOCATE,1/1/2017 10:30,1030,HOTEL / MOTEL / ETC.,78753.0,1,240.0,NaN,Aggravated Assault,"(30.35979391, -97.688358)"
1,AGG ASSAULT,1/1/2017 0:07,7,STREETS / HWY / ROAD / ALLEY,78701.0,2,381.0,NaN,Aggravated Assault,"(30.26816195, -97.74136237)"
2,AGG ASSAULT,1/1/2017 4:20,420,PARKING LOTS / GARAGE,78723.0,3,294.0,NaN,Aggravated Assault,"(30.3186038, -97.68793685)"
3,AGG ASSAULT FAM/DATE VIOLENCE,1/1/2017 0:00,0,RESIDENCE / HOME,78745.0,3,526.0,NaN,Aggravated Assault,"(30.1960291, -97.79102086)"
4,AGG FORCED SODOMY,1/1/2017 21:29,2129,RESIDENCE / HOME,78753.0,7,241.0,NaN,Rape,NaN


In [6]:
Highest_Offense_Description = Newdata['Highest Offense Description'].value_counts()
Highest_Offense_Description

#Newdata = [[Highest_Offense_Description]]

#Newdata = pd.DataFrame(data_df,columns = ["Highest Offense Description"])
# Newdata

FAMILY DISTURBANCE                25728
BURGLARY OF VEHICLE               23572
THEFT                             18444
CRIMINAL MISCHIEF                 12923
ASSAULT W/INJURY-FAM/DATE VIOL     8100
                                  ...  
VIOL CIVIL RIGHTS PRISONER            1
VIOL PO / SEXUAL ASLT VICTIM          1
CRASH/MANSLAUGHTER                    1
COMMERCIAL BRIBERY                    1
VIOL STATE MASSAGE REGULATIONS        1
Name: Highest Offense Description, Length: 361, dtype: int64

In [9]:
range_time_crime = Newdata.loc[(Newdata['Occurred Time']> 1800) & (Newdata['Occurred Time']< 2400)]
range_time_crime.sort_values(by =['Occurred Time'],ascending=True) 
                              

,Highest Offense Description,Occurred Date Time,Occurred Time,Location Type,Zip Code,APD District,PRA,Census Tract,Category Description,Location
207505,THEFT,8/19/2020 18:01,1801,RESIDENCE / HOME,78753.0,7,241.0,NaN,Theft,"(30.35804315, -97.67232957)"
176456,DATING DISTURBANCE,3/23/2020 18:01,1801,RESIDENCE / HOME,78705.0,2,358.0,NaN,NaN,"(30.28432076, -97.74792574)"
134223,CHILD CUSTODY INTERFERE,8/22/2019 18:01,1801,RESIDENCE / HOME,78754.0,6,206.0,NaN,NaN,"(30.40514252, -97.64220838)"
25808,ASSAULT BY CONTACT,7/4/2017 18:01,1801,STREETS / HWY / ROAD / ALLEY,78757.0,7,248.0,NaN,NaN,"(30.36323538, -97.71763016)"
165060,FAMILY DISTURBANCE,1/20/2020 18:01,1801,RESIDENCE / HOME,78717.0,5,573.0,NaN,NaN,"(30.47625073, -97.78035192)"
...,...,...,...,...,...,...,...,...,...,...
179266,CRIMINAL TRESPASS,4/10/2020 23:59,2359,PARKING LOTS / GARAGE,78724.0,7,302.0,NaN,NaN,"(30.2996365, -97.64546733)"
19027,BURGLARY OF VEHICLE,5/17/2017 23:59,2359,RESIDENCE / HOME,78741.0,3,442.0,NaN,Theft,"(30.23496305, -97.7183772)"
105805,CRED CARD ABUSE - OTHER,3/17/2019 23:59,2359,GAS / SERVICE STATIONS,78701.0,1,844.0,NaN,NaN,"(30.26681904, -97.73467173)"
11777,BURGLARY OF RESIDENCE,3/27/2017 23:59,2359,RESIDENCE / HOME,78724.0,7,302.0,NaN,Burglary,"(30.29851178, -97.62756827)"
